<a href="https://colab.research.google.com/github/Abdullah-Sada/AI-Assignment/blob/main/TensorFlow_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[Problem 1] Looking back on scratch



*   I had to initialize the weights
*   Forward propagation matrix calculations were required



*   Matrix calculations for back propagation were required.
*   I needed an epoch loop

*   Losses had to be calculated.









In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Problem 1: Recap of Scratch Implementation

# Step 1: Initialize weights and biases
def initialize_parameters(n_input, n_hidden1, n_hidden2, n_classes):
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    return weights, biases

# Step 2: Implement forward pass
def forward_propagation(X, weights, biases):
    layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    output_layer = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return output_layer

# Step 3: Implement loss function
def compute_loss(logits, Y):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
    return loss

# Step 4: Implement optimization algorithm
def optimize(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss)
    return train_op

# Step 5: Implement mini-batch iteration
class GetMiniBatch:
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0] / self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self, item):
        p0 = item * self.batch_size
        p1 = item * self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter * self.batch_size
        p1 = self._counter * self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

#[Problem 2] Consider the correspondence between scratch and TensorFlow

In [2]:
!pip install tensorflow==2.8

In [3]:
import numpy as np
import tensorflow as tf
tf = tf.compat.v1
tf.disable_eager_execution()

In [5]:
import pandas as pd

# Load dataset
try:
    df = pd.read_csv("Iris.csv")
    print("Dataset loaded successfully")
except FileNotFoundError:
    print("File not found. Please check the file path and name.")

# Display the first few rows of the dataset
print(df.head())


Dataset loaded successfully
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


In [6]:
"""
Binar classification of Iris dataset using neural network implemented in TensorFlow
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

"""
Don't forget when you change the version of tensorflow to the 1.x series.
Install the GPU with "!pip install tensorflow-gpu==1.14.0".
tf.test.gpu_device_name()でGPUの設定状態を確認し、認識されるかを確認します。
If successful, a log is output; if not recognized, nothing is output。
"""

#Load dataset
df = pd.read_csv("Iris.csv")

#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# Convert to NumPy array
X = np.array(X)
y = np.array(y)
#Convert labels to numbers
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Further split into train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    Iterator to get a mini-batch

    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int64)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    #Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output

#Read network structure
logits = example_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialization of #variable
init = tf.global_variables_initializer()


#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 7.0241, val_loss : 67.6859, acc : 0.375
Epoch 1, loss : 3.4241, val_loss : 23.4026, acc : 0.312
Epoch 2, loss : 1.9387, val_loss : 11.6681, acc : 0.375
Epoch 3, loss : 2.0917, val_loss : 13.1400, acc : 0.312
Epoch 4, loss : 1.7685, val_loss : 17.7284, acc : 0.312
Epoch 5, loss : 1.6097, val_loss : 12.9607, acc : 0.312
Epoch 6, loss : 1.4402, val_loss : 10.0593, acc : 0.312
Epoch 7, loss : 1.3704, val_loss : 9.4797, acc : 0.312
Epoch 8, loss : 1.2536, val_loss : 9.8518, acc : 0.312
Epoch 9, loss : 1.1476, val_loss : 8.5670, acc : 0.375
Epoch 10, loss : 1.0930, val_loss : 8.0429, acc : 0.375
Epoch 11, loss : 1.0412, val_loss : 7.8791, acc : 0.375
Epoch 12, loss : 0.9804, val_loss : 7.1233, acc : 0.375
Epoch 13, loss : 0.9326, val_loss : 6.7908, acc : 0.375
Epoch 14, loss : 0.8792, val_loss : 6.2493, acc : 0.375
Epoch 15, loss : 0.8304, val_loss : 5.7680, acc : 0.375
Epoch 16, loss : 0.7835, val_loss : 5.2886, acc : 0.438
Epoch 17, loss : 0.7384, val_loss : 4.8037, acc : 0

#[Problem 3] Create a model of Iris using all three types of objective variables

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Load dataset
df = pd.read_csv("Iris.csv")

# Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# Convert to NumPy array
X = np.array(X)
y = np.array(y)
# Convert labels to numbers
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Further split into train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int64)
    def __len__(self):
        return self._stop
    def __getitem__(self, item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# Determine the shape of the argument to be passed to the calculation graph
X_ph = tf.placeholder("float", [None, n_input])
Y_ph = tf.placeholder("float", [None, n_classes])

# Train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    tf.random.set_random_seed(0)
    # Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output

# Read network structure
logits = example_net(X_ph)

# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y_ph, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated result
correct_pred = tf.equal(tf.sign(Y_ph - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialization of variables
init = tf.global_variables_initializer()

# Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X_ph: mini_batch_x, Y_ph: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X_ph: mini_batch_x, Y_ph: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X_ph: X_val, Y_ph: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X_ph: X_test, Y_ph: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 2.5685, val_loss : 19.7636, acc : 0.375
Epoch 1, loss : 0.4319, val_loss : 2.8135, acc : 0.750
Epoch 2, loss : 0.5106, val_loss : 0.7691, acc : 0.812
Epoch 3, loss : 0.0792, val_loss : 2.5278, acc : 0.500
Epoch 4, loss : 0.1210, val_loss : 0.6748, acc : 0.812
Epoch 5, loss : 0.0096, val_loss : 0.1199, acc : 0.938
Epoch 6, loss : 0.0215, val_loss : 0.0272, acc : 1.000
Epoch 7, loss : 0.0045, val_loss : 0.0282, acc : 1.000
Epoch 8, loss : 0.0059, val_loss : 0.0259, acc : 1.000
Epoch 9, loss : 0.0039, val_loss : 0.0121, acc : 1.000
Epoch 10, loss : 0.0024, val_loss : 0.0215, acc : 1.000
Epoch 11, loss : 0.0028, val_loss : 0.0118, acc : 1.000
Epoch 12, loss : 0.0022, val_loss : 0.0104, acc : 1.000
Epoch 13, loss : 0.0020, val_loss : 0.0114, acc : 1.000
Epoch 14, loss : 0.0018, val_loss : 0.0123, acc : 1.000
Epoch 15, loss : 0.0018, val_loss : 0.0107, acc : 1.000
Epoch 16, loss : 0.0017, val_loss : 0.0102, acc : 1.000
Epoch 17, loss : 0.0016, val_loss : 0.0105, acc : 1.000
E

#[Problem 4] Create a model of House Prices

In [9]:
#Load dataset
house_df = pd.read_csv("Housing_train.csv")
#Condition extraction from data frame
house_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
_X = house_df[["GrLivArea", "YearBuilt"]].to_numpy()
_y = house_df[["SalePrice"]].to_numpy()

In [11]:
_X.shape, _y.shape

((1460, 2), (1460, 1))

In [12]:
"""
Binar classification of Iris dataset using neural network implemented in TensorFlow
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

"""
Don't forget when you change the version of tensorflow to the 1.x series.
Install the GPU with "!pip install tensorflow-gpu==1.14.0".
tf.test.gpu_device_name()でGPUの設定状態を確認し、認識されるかを確認します。
If successful, a log is output; if not recognized, nothing is output。
"""






#Split into train and test
X_train3, X_test3, y_train3, y_test3 = train_test_split(_X, _y, test_size=0.2, random_state=0)
# Further split into train and val
X_train3, X_val3, y_train3, y_val3 = train_test_split(X_train3, y_train3, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    Iterator to get a mini-batch

    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int64)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train3.shape[1]
n_samples = X_train3.shape[0]
n_classes = 1

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# train mini batch iterator
get_mini_batch_train3 = GetMiniBatch(X_train3, y_train3, batch_size=batch_size)
def estimation_calc(X,Y, X_train3, y_train3,X_val3, Y_val3,X_test3, y_test3,num_epochs, get_mini_batch_train3):
  def example_net(x):
      """
      Simple 3-layer neural network
      """
      tf.random.set_random_seed(0)
      #Declaration of weights and biases
      weights = {
          'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
          'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
          'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
      }
      biases = {
          'b1': tf.Variable(tf.random_normal([n_hidden1])),
          'b2': tf.Variable(tf.random_normal([n_hidden2])),
          'b3': tf.Variable(tf.random_normal([n_classes]))
      }

      layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
      layer_1 = tf.nn.relu(layer_1)
      layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
      layer_2 = tf.nn.relu(layer_2)
      layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
      return layer_output

#Read network structure
logits = example_net(X)

# Objective function
loss_op = tf.losses.mean_squared_error(labels=Y, predictions=logits)
#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
pred = tf.keras.activations.linear(logits)

#Initialization of #variable
init = tf.global_variables_initializer()


#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train3.shape[0]/batch_size).astype(np.int64)
        total_loss = []
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train3):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run([loss_op], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss.append(loss)
        val_loss = sess.run([loss_op], feed_dict={X: X_val3, Y: y_val3})
        #print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss))
    #test_acc = sess.run(accuracy, feed_dict={X: X_test2, Y: y_test2})
    print("test_Mse : {:.3f}".format(np.sum(total_loss)))

test_Mse : 286924505088.000


#[Problem 5] Create a model of MNIST

In [13]:
#MNIST

In [14]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

In [15]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

11501568/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

In [16]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
X_train /= 255
X_test /= 255


X_train4, X_test4, y_train4, y_test4 = train_test_split(X_train, y_train, test_size=0.2)

In [17]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train4[:, np.newaxis])
y_test_one_hot = enc.fit_transform(y_test4[:, np.newaxis])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train4.shape[1]
n_samples = X_train4.shape[0]
n_classes = 10

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# train mini batch iterator
get_mini_batch_train4 = GetMiniBatch(X_train4, y_train_one_hot, batch_size=batch_size)
def estimation_calc(X,Y, X_train4, y_train4,X_val4, Y_val4,X_test4, y_test4,num_epochs, get_mini_batch_train4):
  def example_net(x):
      """
      Simple 3-layer neural network
      """
      tf.random.set_random_seed(0)
      #Declaration of weights and biases
      weights = {
          'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
          'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
          'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
      }
      biases = {
          'b1': tf.Variable(tf.random_normal([n_hidden1])),
          'b2': tf.Variable(tf.random_normal([n_hidden2])),
          'b3': tf.Variable(tf.random_normal([n_classes]))
      }

      layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
      layer_1 = tf.nn.relu(layer_1)
      layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
      layer_2 = tf.nn.relu(layer_2)
      layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
      return layer_output

#Read network structure
logits = example_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
pred = logits
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialization of #variable
init = tf.global_variables_initializer()


#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train4.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train4):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss +=loss
        total_loss /= n_samples
        print("Epoch {}, loss: {:.4f}, acc: {:.3f}".format(epoch, total_loss, acc))

    test_acc = sess.run(accuracy, feed_dict={X: X_test4, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Epoch 0, loss: 2.7853, acc: 0.700
Epoch 1, loss: 0.5054, acc: 0.600
Epoch 2, loss: 0.2449, acc: 0.500
Epoch 3, loss: 0.1383, acc: 0.600
Epoch 4, loss: 0.0886, acc: 0.700
Epoch 5, loss: 0.0643, acc: 0.700
Epoch 6, loss: 0.0492, acc: 0.800
Epoch 7, loss: 0.0402, acc: 0.800
Epoch 8, loss: 0.0340, acc: 0.800
Epoch 9, loss: 0.0296, acc: 0.800
test_acc : 0.897
